### Exploring data from datahub.io

Acquiring data from [here](https://github.com/datasets/covid-19) and maybe soon be able to automate the integration of that data with wikidata.

Just some things to think about (jvfe):
- How to properly reference the data? Choose [datahub.io](https://datahub.io/core/covid-19) as the reference?
    - They aggregate it from various sources
    
    
- I've acquired the country outbreak items via the following query and modified it slightly to better merge the items.
```
SELECT ?item ?itemLabel ?countryid ?countryidLabel
WHERE 
{
  ?item p:P31 ?statement. 
      ?statement ps:P31 wd:Q3241045. 
      ?statement pq:P642 wd:Q84263196.
      ?statement pq:P3005 ?countryid.
      ?countryid wdt:P31 wd:Q6256.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
```

In [15]:
import pandas as pd

In [16]:
from datetime import date, time, timedelta
yesterday = date.today() - timedelta(days=1)
today = date.today()

yesterday_table = yesterday.strftime("%Y-%m-%d")
today_table = today.strftime("%Y-%m-%d")


In [17]:
countries = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv")
wdt_items = pd.read_csv("../data/country_outbreaks.csv")

In [18]:
full = pd.merge(countries, wdt_items, on="Country")
full

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid
0,2020-01-22,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
1,2020-01-23,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
2,2020-01-24,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
3,2020-01-25,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
4,2020-01-26,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
...,...,...,...,...,...,...,...,...
12735,2020-04-24,Zimbabwe,29,2,4,Q88164033,2020 coronavirus pandemic in Zimbabwe,Q954
12736,2020-04-25,Zimbabwe,31,2,4,Q88164033,2020 coronavirus pandemic in Zimbabwe,Q954
12737,2020-04-26,Zimbabwe,31,2,4,Q88164033,2020 coronavirus pandemic in Zimbabwe,Q954
12738,2020-04-27,Zimbabwe,32,5,4,Q88164033,2020 coronavirus pandemic in Zimbabwe,Q954


In [50]:
#Most recent data seems to be from the day before

query = "Date == @yesterday_table"
recent = full.query(query)

## that does not happen aways, though.


dates_in_full = [datetime.strptime(date, "%Y-%m-%d") for date in full["Date"]]
most_recent_date = max(dates_in_recent).strftime("%Y-%m-%d")

# pd.query did not work
recent = full[full["Date"] == most_recent_date ]

recent.head()

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid
97,2020-04-28,Afghanistan,1828,228,58,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
195,2020-04-28,Algeria,3649,1651,437,Q87202921,2020 coronavirus pandemic in Algeria,Q262
293,2020-04-28,Angola,27,6,2,Q88082534,2020 coronavirus pandemic in Angola,Q916
391,2020-04-28,Antigua and Barbuda,24,11,3,Q87708331,2020 coronavirus pandemic in Antigua and Barbuda,Q781
489,2020-04-28,Argentina,4127,1162,207,Q87235137,2020 coronavirus pandemic in Argentina,Q414


In [51]:
# The following countries appear to be updated manually from more specific sources.
idx = recent['Country'].isin(['US', 'United Kingdom', 'France', 'Sweden', 'Brazil', 'Netherlands',
                             'China', 'Italy', 'Spain', 'Germany', 'Iran', 'Mexico', 'Argentina',
                             'Canada', 'Spain', 'Norway', 'Portugal', 'Tunisia', 'Uruguay'])
not_manual = recent[~idx]

In [52]:
yesterday_wdt = yesterday.strftime("+%Y-%m-%dT00:00:00Z/11")
today_wdt = today.strftime("+%Y-%m-%dT00:00:00Z/11")

with open(f'../data/{today_table}.qs', 'w') as file:
    for index, row in not_manual.iterrows():
        print(
              row['item'] + "|P1603|" + str(int(row['Confirmed'])) + "|P585|" + yesterday_wdt + "|S854|" + '"' + 
                    "https://github.com/datasets/covid-19" + '"' +
                    "|S813|" + today_wdt + "\n" +
              row['item'] + "|P1120|" + str(int(row['Deaths'])) + "|P585|" + yesterday_wdt + "|S854|" + '"' + 
                    "https://github.com/datasets/covid-19" + '"' +
                    "|S813|" + today_wdt + "\n" +
              row['item'] + "|P8010|" + str(int(row['Recovered'])) + "|P585|" + yesterday_wdt + "|S854|" + '"' + 
                    "https://github.com/datasets/covid-19" + '"' +
                    "|S813|" + today_wdt + "\n",
                file = file)

In [53]:
%run check_last_update_for_country_items.py

In [54]:
country_outbreak_items_of_interest = list(recent["item"])

# Api only takes 50 at a time, so we have to cut it.


# implementation from    https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def get_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

        
chunks_of_country_outbreak_items_of_interest = list(get_chunks(country_outbreak_items_of_interest, 50))        
        
outbreak_item_to_timestamp = {}

for chunk in chunks_of_country_outbreak_items_of_interest:
    outbreak_item_to_timestamp.update(get_timestamp_of_last_edits(chunk))


In [55]:
recent["timestamp_of_last_edit"] = recent["item"].map(outbreak_item_to_timestamp)

recent.head(1)

/home/lubianat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit
97,2020-04-28,Afghanistan,1828,228,58,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889,2020-04-30T02:06:00Z


In [57]:
from datetime import datetime

def convert_timestamp_to_time_until_now(timestamp):

    time_in_datetime_format = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
    diff = datetime.now() - time_in_datetime_format
    return(diff)

In [58]:
recent["time_from_last_edit_until_now"] = recent["timestamp_of_last_edit"].map(convert_timestamp_to_time_until_now)

recent.head(1)

/home/lubianat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit,time_from_last_edit_until_now
97,2020-04-28,Afghanistan,1828,228,58,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889,2020-04-30T02:06:00Z,04:47:22.717114


In [59]:
outdated_items = recent[recent["time_from_last_edit_until_now"] > timedelta(hours=23)]

In [60]:
outdated_items.head(5)

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit,time_from_last_edit_until_now
293,2020-04-28,Angola,27,6,2,Q88082534,2020 coronavirus pandemic in Angola,Q916,2020-04-27T10:08:48Z,2 days 20:44:34.717152
391,2020-04-28,Antigua and Barbuda,24,11,3,Q87708331,2020 coronavirus pandemic in Antigua and Barbuda,Q781,2020-04-23T21:42:43Z,6 days 09:10:39.717164
587,2020-04-28,Australia,6744,5665,89,Q83873548,2020 coronavirus pandemic in Australia,Q408,2020-04-27T03:48:57Z,3 days 03:04:25.717184
1077,2020-04-28,Benin,64,33,1,Q87781572,2020 coronavirus pandemic in Benin,Q962,2020-04-23T11:38:16Z,6 days 19:15:06.717233
1175,2020-04-28,Bhutan,7,5,0,Q87715166,2020 coronavirus pandemic in Bhutan,Q917,2020-04-28T14:59:06Z,1 days 15:54:16.717243


In [64]:
dates_in_full = [datetime for date in full["Date"]]
most_recent_date = max(dates_in_recent).strftime("%Y-%m-%d")


table_date_in_wikidata_format = datetime.strptime(outdated_items["Date"].values[0], "%Y-%m-%d").strftime("+%Y-%m-%dT00:00:00Z/11")

point_in_time = "|P585|" +  table_date_in_wikidata_format

today_wdt = today.strftime("+%Y-%m-%dT00:00:00Z/11")


reference_URL = "|S854|" + '"' +  "https://datahub.io/core/covid-19" + '"'
retrieved_in  = "|S813|" + today_wdt 
filename_in_archive = "|S7793|" + '"' +  "r/countries-aggregated.csv" + '"'

reference = reference_URL +  retrieved_in + filename_in_archive


with open(f'../data/{today_table}_outdated_items.qs', 'w') as file:
    for index, row in not_manual.iterrows():
        print(
              row['item'] + "|P1603|" + str(int(row['Confirmed'])) + point_in_time + reference + "\n" +
              row['item'] + "|P1120|" + str(int(row['Deaths'])) + point_in_time + reference + "\n" +
              row['item'] + "|P8010|" + str(int(row['Recovered'])) + point_in_time + reference                    + "\n",
                file = file)